In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,220 kB]
Get:14 http://sec

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProject").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

dfShoes = spark.read.option("sep", "\t").csv("file://" + SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), header=True)
dfShoes.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [6]:
dropna_df = dfShoes.dropna(how='any')
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [7]:
dfShoesCleaned = dropna_df.dropDuplicates(['product_id'])
dfShoesCleaned.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28419055|R2A4N5UIA3H3ZO|0456787283|     163175355|FX1 Small Adult A...|           Shoes|          1|            1|          1|   N|                Y|     still too large|I bought these wi...| 2013-03-16|
|         US|   35303306| RU03YWQYD7VIN|345200015X|     935911590|Mirage Sunglasses...|           Shoes|          5|    

In [8]:
rows = dfShoesCleaned.count()
rows

1900895

In [9]:
dfShoesCleaned = dfShoesCleaned.dropDuplicates(['customer_id'])
dfShoesCleaned.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10000018|R3SFRG4CC8CT57|B009YSYE06|       6815942|Report Women's Le...|           Shoes|          3|            0|          0|   N|                Y|           Just okay|I must say these ...| 2013-10-23|
|         US|   10000144|R1WZNG45ZG9IRL|B00FAKP9M4|     376417938|Easy Street Women...|           Shoes|          4|    

In [10]:
rows2 = dfShoesCleaned.count()
rows2

1364720

In [11]:
review_id_table_shoes = dfShoesCleaned.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_table_shoes.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3SFRG4CC8CT57|   10000018|B009YSYE06|       6815942| 2013-10-23|
|R36L1VQP51OMOU|   10000023|B002WRGVV6|     651010533| 2012-03-21|
|R2L4WAOM758QSD|   10000064|B008B1ZDC0|     569257823| 2013-06-17|
|R2NFLNPUCGZR2G|    1000011|B004LF3MZ0|     148862416| 2014-10-16|
|R2TE4JIZBYZYDG|   10000127|B007R0TFHA|     184377633| 2013-01-19|
|R2WPRQYEQN3N88|   10000128|B000FBRR14|     675121801| 2010-12-08|
|R35B6YD5GGCFCH|   10000141|B003INN6RK|     469558458| 2012-01-09|
|R1WZNG45ZG9IRL|   10000144|B00FAKP9M4|     376417938| 2014-10-13|
|R157L8L3MYG9P7|    1000022|B00PVHJRZG|     407241419| 2015-01-17|
|R31V4JDBVFP1AX|   10000345|B006ZF39KW|      66656225| 2013-08-16|
| RR6P4IIPIQOOF|   10000387|B00BLFL34K|     240876511| 2014-03-28|
|R1Y2NY9RTJAPO0|   10000393|B000GCAXKE|     226584928| 2013-11

In [12]:
products_shoes = dfShoesCleaned.select(["product_id","product_title"])
products_shoes.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B009YSYE06|Report Women's Le...|
|B002WRGVV6|Nine West Women's...|
|B008B1ZDC0|Sanuk Men's Chum ...|
|B004LF3MZ0|Nike Men's Air Ma...|
|B007R0TFHA|Wanted Shoes Wome...|
|B000FBRR14|adidas Men's Smit...|
|B003INN6RK|GUESS Women's Ade...|
|B00FAKP9M4|Easy Street Women...|
|B00PVHJRZG|Coach Darcy Leath...|
|B006ZF39KW|Southpole DIESEL ...|
|B00BLFL34K|SeaVees Men's 12-...|
|B000GCAXKE|Deer Stags Kids B...|
|B00JGNGONY|FURLA Metropolis ...|
|B00W0S2RMY|GUESS Women's Ely...|
|B00HHYFND6|Mizuno Men's Wave...|
|B00774FXH2|Bass Men's Larson...|
|B008JE51DK|Dansko Women's Pi...|
|B0085DA7E8|Teva Women's Numa...|
|B005AL96BI|Diane von Fursten...|
|B007S0DCVO|Osiris NYC 83 Ska...|
+----------+--------------------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import monotonically_increasing_id 

dfShoesCleaned_shoes = dfShoesCleaned.select("*").withColumn("customer_count", monotonically_increasing_id())
dfShoesCleaned_shoes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|customer_count|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------+
|         US|   10000018|R3SFRG4CC8CT57|B009YSYE06|       6815942|Report Women's Le...|           Shoes|          3|            0|          0|   N|                Y|           Just okay|I must say these ...| 2013-10-23|             0|
|         US|   10000144|R1WZNG45ZG9IRL|B00FAKP9M4|     3764

In [16]:
customers_shoes = dfShoesCleaned_shoes.select(["customer_id","customer_count"])
customers_shoes.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   39346768|             0|
|   31973704|             1|
|   43220439|             2|
|   18401902|             3|
|   30373645|             4|
|   49949641|             5|
|   14551404|             6|
|   11640490|             7|
|   39212844|             8|
|   14426653|             9|
|   26632941|            10|
|   18070317|            11|
|   51391946|            12|
|   41334994|            13|
|   39093279|            14|
|   25769599|            15|
|   52573642|            16|
|   29903273|            17|
|   45717823|            18|
|   52809622|            19|
+-----------+--------------+
only showing top 20 rows



In [17]:
vine_table_shoes = dfShoesCleaned_shoes.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table_shoes.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3SFRG4CC8CT57|          3|            0|          0|   N|
|R36L1VQP51OMOU|          2|            0|          0|   N|
|R2L4WAOM758QSD|          5|            2|          2|   N|
|R2NFLNPUCGZR2G|          1|            0|          8|   N|
|R2TE4JIZBYZYDG|          4|            0|          0|   N|
|R2WPRQYEQN3N88|          2|            3|          4|   N|
|R35B6YD5GGCFCH|          5|            0|          0|   N|
|R1WZNG45ZG9IRL|          4|            1|          1|   N|
|R157L8L3MYG9P7|          4|            1|          2|   N|
|R31V4JDBVFP1AX|          5|            0|          0|   N|
| RR6P4IIPIQOOF|          2|            0|          1|   N|
|R1Y2NY9RTJAPO0|          5|            2|          2|   N|
| RB1AXT9CIBF4T|          5|            0|          0|   N|
|R31E6LGZHNYBVL|          5|            

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}


In [ ]:
review_id_table_shoes.write.jdbc(url=jdbc_url, table='review_id_table_shoes', mode=mode, properties=config)

In [ ]:
products_shoes.write.jdbc(url=jdbc_url, table='products_shoes', mode=mode, properties=config)

In [ ]:
customers_shoes.write.jdbc(url=jdbc_url, table='customers_shoes', mode=mode, properties=config)

In [ ]:
vine_table_shoes.write.jdbc(url=jdbc_url, table='vine_table_shoes', mode=mode, properties=config)